In [6]:
from ultralytics import YOLO

# Burada kullanacağımız modeli seçiyoruz ve model değişkenine atıyoruz

model= YOLO("yolov8m.pt") 

In [7]:
# model değişkeni predict metoduyla resim üzerinde nesne tahmini yapıyor

# save=True: Tahmin sonuçlarını kaydetme opsiyonunu belirtir.

# verbose=False Kodun çalıştırılması sırasında ekran üzerinde detaylı çıktıları gösterip göstermemeyi belirtir. 
# Sadece temel çıktıları görmeyi sağlar.

# sonuc değişkeni üzerinden çıktıya ve çıktı özelliklerine ulaşılır

sonuc=model.predict(source='kedi.jpg',save=True,verbose=False) 

Results saved to runs\detect\predict9


In [8]:
# sonuc[0] ->üzerinde tahmin yapılan ilk çıktı

# sonuc[0].boxes -> çıktıya ait bounding box'ları (çerçeveleri) içeren özellikleri ifade eder.

# sonuc[0].boxes.xyxy -> bounding box'ların(çerçevelerin) koordinatlarını ifade eder. 

sonuc[0].boxes.xyxy

tensor([[ 65.2453,  93.8643, 360.7711, 572.9806]])

In [9]:
x1,y1,x2,y2=sonuc[0].boxes.xyxy[0] #kordinat değerlerini çoklu atama ile 4 değişkene atıyoruz

In [10]:
x1 #sol üst x kordinatı

tensor(65.2453)

In [11]:
y1 #sol üst y kordinatı

tensor(93.8643)

In [12]:
x2 #sağ alt x kordinatı

tensor(360.7711)

In [13]:
y2 #sağ alt y kordinatı

tensor(572.9806)

In [14]:
skor=sonuc[0].boxes.conf[0] #boxes ->çerçevenin özelliklerini içerir  // conf[0] -> güven skorunu içerir,ilk skoru seçiyor
skor  #doğruluk oranı,skoru


tensor(0.9532)

In [15]:
etiket=sonuc[0].boxes.cls[0] #boxes ->çerçevenin özelliklerini içerir // cls[0] -> sınıf etiketini içerir,ilk etiketi seçer

etiket #burada resimde tespit edilen nesnenin coco8.yaml'daki atanmış idsine erişiyoruz. kedi->15

tensor(15.)

In [16]:
#yaptığımız örneklerde tanımlanan nesnelerin ismi bu listedeki değerlerden olucak

etiketler=['insan','bisiklet','araba','motorsiklet','uçak','otobüs','tren','kamyon','bot','trafik lambası','yangın musluğu',
'dur işareti','park metre','bank','kus','kedi','kopek','at','koyun','inek','fil','ayı','zebra','zurafa','sırt çantası','şemsiye',
'el çantası','kravat','bavul','frizbi','kayaklar','snowbord','spor topu','uçurtma','beyzbol sopası','beyzbol eldiveni','kaykay',
'sörf tahtası','tenis raketi','şişe','şarap bardağı','bardak','çatal','bıçak','kaşık','kase','muz','elma','sandviç','portakal',
'brokoli','havuç','hotdog','pizza','donut','pasta','sandalye','kanepe','saksı bitkisi','yatak','yemek masası','tuvalet',
'televizyon','laptop','maus','uzaktan kumanda','klavye','cep telefonu','mikrodalga','fırın','tost makinesi','lavabo','buzdolabı',
'kitap','saat','vazo','makas''oyuncak ayı','saç kurutucu','diş fırçası']

In [17]:
canli=['insan','kus','kedi','kopek','at','koyun','inek','fil','ayı','zebra','zurafa'] #canlı,cansız ayrımında kullanılıcak liste

# Resimde Nesne Tespiti 


In [18]:
import cv2

resim=cv2.imread('ikinci.jpg') #resim yükleme kısmı

""" (1) OpenCV kütüphanesini kullanarak BGR (Blue, Green, Red) renk uzayında temsil edilen bir görüntüyü RGB (Red, Green, Blue)
renk uzayına dönüştürüyoruz.Resim RGB formatında oluyor ve bu formatı modelle tahminleyerek sonuc değişkenine atıyoruz """

rgb_resim=cv2.cvtColor(resim,cv2.COLOR_BGR2RGB) 
sonuc = model(rgb_resim) 

font = cv2.FONT_HERSHEY_SIMPLEX #yazı türünü font değişkenine atadık

""" (2) for döngüsü sayesinde resimde tespit edilen nesnelerin değerlerine erişicez   """

for i in range(len(sonuc[0].boxes)): # len(sonuc[0].boxes ->resimde tespit edilen nesnelerin sayısı
    
    x1,y1,x2,y2=sonuc[0].boxes.xyxy[i] #resimdeki nesnelerin kordinatları (x1 ve y1->sol üst , x2 ve y2->sağ alt)
    skor=sonuc[0].boxes.conf[i] #doğruluk oranı
    etiket=sonuc[0].boxes.cls[i]  #etiket idsi coco8.yaml veri setine göre
    
    x1,y1,x2,y2,skor,etiket=int(x1),int(y1),int(x2),int(y2),float(skor),int(etiket) #sırayla değerleri atama
    
    nesne_ismi=etiketler[etiket] #etiketler->bizim isimlendirmelerimiz için kullanılacak liste/#etiket->nesnenin idsini dönücek
    
    """(3) nesne_ismi değişkeni sayesinde canlı,cansız ayrımı yapabilcez ve nesne_ismi bizim verdiğimiz değerleri alıcak""" 
    
    if skor<0.2: #doğruluk oranı 0.2 den düşükse değerlendirmeden geç 
        continue
        
    if nesne_ismi in canli: #tespit edilen nesne canlı listesinde ise...
        cv2.rectangle(resim,(x1,y1),(x2,y2),(29,94,222),2) #nesne etrafında oluşucak dikdörtgenin kordinatı,rengi ve kalınlığı
        yazi= nesne_ismi+' '+str(format(skor, '.2f'))+' '+'canli' #nesnenin üstünde yazıcak yazı 
        cv2.putText(resim, yazi,(x1, y1-10), font, 0.75,(255,0,0),2) #nesnenin üzerinde yazıcak yazıyı ayarlama
        
    if nesne_ismi not in canli:#canlı listesinde olmayan nesne ise...
        cv2.rectangle(resim,(x1,y1),(x2,y2),(29,222,55),2)
        yazi= nesne_ismi+' '+str(format(skor, '.2f'))+' '+'cansiz'
        cv2.putText(resim, yazi,(x1, y1-10), font, 0.55,(255,0,255),2) #nesnenin üzerinde yazıcak yazıyı ayarlama
        
  
cv2.imshow("kamera",resim) # en son hali gösterme, pencerede kamera ismi yazıcak

cv2.waitKey() #belirli süre bekletir klavyeden tuşa tıklanınca pencere kapanır
cv2.destroyAllWindows() #pencereyi kapatan metot

# Resimde Spesifik Bir Nesneyi (Sınıfı) Gösterme


In [19]:
import cv2

resim=cv2.imread('ikinci.jpg') #resim yükleme kısmı

rgb_resim=cv2.cvtColor(resim,cv2.COLOR_BGR2RGB) 
sonuc = model(rgb_resim) 

font = cv2.FONT_HERSHEY_SIMPLEX #yazı türünü font değişkenine atadık

""" (2) for döngüsü sayesinde resimde tespit edilen nesnelerin değerlerine erişicez   """

for i in range(len(sonuc[0].boxes)): # len(sonuc[0].boxes ->resimde tespit edilen nesnelerin sayısı
    
    x1,y1,x2,y2=sonuc[0].boxes.xyxy[i] #resimdeki nesnelerin kordinatları (x1 ve y1->sol üst , x2 ve y2->sağ alt)
    skor=sonuc[0].boxes.conf[i] #doğruluk oranı
    etiket=sonuc[0].boxes.cls[i]  #etiket idsi coco8.yaml veri setine göre
    
    x1,y1,x2,y2,skor,etiket=int(x1),int(y1),int(x2),int(y2),float(skor),int(etiket) #sırayla değerleri atama
    
    nesne_ismi=etiketler[etiket] #etiketler->bizim isimlendirmelerimiz için kullanılacak liste/#etiket->nesnenin idsini dönücek
        
    if skor<0.2 or nesne_ismi!='bank': #doğruluk oranı 0.2 den düşükse ve tespit edilen nesne ismi bank değilse geç 
        continue
        
    if nesne_ismi in canli: #tespit edilen nesne canlı listesinde ise...
        cv2.rectangle(resim,(x1,y1),(x2,y2),(29,94,222),2) #nesne etrafında oluşucak dikdörtgenin kordinatı,rengi ve kalınlığı
        yazi= nesne_ismi+' '+str(format(skor, '.2f'))+' '+'canli' #nesnenin üstünde yazıcak yazı 
        cv2.putText(resim, yazi,(x1, y1-10), font, 0.75,(255,0,0),2) #nesnenin üzerinde yazıcak yazıyı ayarlama
        
    if nesne_ismi not in canli: #canlı listesinde olmayan nesne ise...
        cv2.rectangle(resim,(x1,y1),(x2,y2),(29,222,55),2)
        yazi= nesne_ismi+' '+str(format(skor, '.2f'))+' '+'cansiz'
        cv2.putText(resim, yazi,(x1, y1-10), font, 0.55,(255,0,255),2) #nesnenin üzerinde yazıcak yazıyı ayarlama
        
  
cv2.imshow("kamera",resim) # en son hali gösterme, pencerede kamera ismi yazıcak

cv2.waitKey() #belirli süre bekletir klavyeden tuşa tıklanınca pencere kapanır
cv2.destroyAllWindows() #pencereyi kapatan metot

# Videoda Nesne Tespiti

In [20]:
import cv2

kamera= cv2.VideoCapture('video1.mp4') #videoyu atama
font = cv2.FONT_HERSHEY_SIMPLEX

while True:

    """ ret ->arenin başarılı bir şekilde okunup okunmadığını belirtir. Eğer kare okunmuşsa, ret değeri True olur """
    """ kare ->okunan kareyi temsil eder ve bu değişken üzerinden işlemler yapılır """
    ret,kare=kamera.read() 
    
    resimler=cv2.cvtColor(kare,cv2.COLOR_BGR2RGB) #videoda elde edilen resimler rgb formatına çevriliyor
    sonuclar = model(resimler,verbose=False) #modelle tahmin edilme işlemi oluyor
    
    for i in range(len(sonuclar[0].boxes)):
        x1,y1,x2,y2=sonuclar[0].boxes.xyxy[i] #resimdeki nesnelerin kordinatları (x1 ve y1->sol üst , x2 ve y2->sağ alt)
        skor=sonuclar[0].boxes.conf[i] #tahminin doğruluk oranı
        etiket=sonuclar[0].boxes.cls[i] #etiket idsi coco8.yaml veri setine göre
        
        x1,y1,x2,y2,skor,etiket=int(x1),int(y1),int(x2),int(y2),float(skor),int(etiket) #sırayla değerleri atama
        
        nesne_ismi=etiketler[etiket]
        
        if skor<0.5: #doğruluk oranı 0.5 den düşükse geç o nesneyi tanıma
            continue

        cv2.rectangle(kare,(x1,y1),(x2,y2),(255,0,0),2) #nesne etrafında oluşucak dikdörtgenin kordinatı,rengi ve kalınlığı
        yazi= nesne_ismi+' '+str(format(skor, '.2f')) #nesnenin üstünde yazıcak yazı 
        cv2.putText(kare, yazi,(x1, y1-10), font, 1,(255,0,0), 2) #nesnenin üzerinde yazıcak yazıyı ayarlama


    cv2.imshow("kamera",kare)
    if cv2.waitKey(1) & 0xFF == ord('q'): #q tuşu ile kapatıcaz öbür türlü kasma oluyor
        break
kamera.release()
cv2.destroyAllWindows()


# Webcam ile Nesne Tespiti 


In [4]:
import cv2

kamera = cv2.VideoCapture(0)  # 0, birinci kamera demektir bilgisayar kamerası yani
font = cv2.FONT_HERSHEY_SIMPLEX


while True:

    """ ret ->karenin başarılı bir şekilde okunup okunmadığını belirtir. Eğer kare okunmuşsa, ret değeri True döner """
    """ kare ->okunan kareyi temsil eder ve bu değişken üzerinden işlemler yapılır """
    ret,kare=kamera.read() 
    
    resimler=cv2.cvtColor(kare,cv2.COLOR_BGR2RGB) #videoda elde edilen resimler rgb formatına çevriliyor
    sonuclar = model(resimler,verbose=False) #modelle tahmin edilme işlemi oluyor
    
    for i in range(len(sonuclar[0].boxes)):
        x1,y1,x2,y2=sonuclar[0].boxes.xyxy[i] #resimdeki nesnelerin kordinatları (x1 ve y1->sol üst , x2 ve y2->sağ alt)
        skor=sonuclar[0].boxes.conf[i] #tahminin doğruluk oranı
        etiket=sonuclar[0].boxes.cls[i] #etiket idsi coco8.yaml veri setine göre
        
        x1,y1,x2,y2,skor,etiket=int(x1),int(y1),int(x2),int(y2),float(skor),int(etiket) #sırayla değerleri atama
        
        nesne_ismi=etiketler[etiket]
        
        if skor<0.5: #doğruluk oranı 0.5 den düşükse geç o nesneyi tanıma
            continue

        cv2.rectangle(kare,(x1,y1),(x2,y2),(255,0,0),2) #nesne etrafında oluşucak dikdörtgenin kordinatı,rengi ve kalınlığı
        yazi= nesne_ismi+' '+str(format(skor, '.2f')) #nesnenin üstünde yazıcak yazı 
        cv2.putText(kare, yazi,(x1, y1-10), font, 1,(255,0,0), 2) #nesnenin üzerinde yazıcak yazıyı ayarlama


    cv2.imshow("kamera",kare)
    if cv2.waitKey(1) & 0xFF == ord('q'): #q tuşu ile kapatıcaz öbür türlü kasma oluyor
        break
kamera.release()
cv2.destroyAllWindows()

# ARDUNIO İLE NESNE TESPİTİ

In [ ]:
from ultralytics import YOLO
import cv2
import serial

# Arduino ile seri iletişim kurmak için kullanılan port ve baud hızı
arduino_port = 'COM7'  # Arduino'nun bağlı olduğu port
baud_rate = 9600  # Arduino ile aynı baud hızı

ser = serial.Serial(arduino_port, baud_rate)

# YOLO modelini başlat
model = YOLO("yolov8m.pt")

# Etiket listesi
etiketler = ['insan', 'bisiklet', 'araba', 'motorsiklet', 'uçak', 'otobüs', 'tren', 'kamyon', 'bot', 'trafik lambası', 'yangın musluğu', 'dur işareti',
          'park metre', 'bank', 'kuş', 'kedi', 'köpek', 'at', 'koyun', 'inek', 'fil', 'ayı', 'zebra', 'zürafa', 'sırt çantası', 'şemsiye', 'el çantası',
          'kravat', 'bavul', 'frizbi', 'kayaklar', 'snowbord', 'spor topu', 'uçurtma', 'beyzbol sopası', 'beyzbol eldiveni', 'kaykay', 'sörf tahtası',
          'tenis raketi', 'şişe', 'şarap bardağı', 'bardak', 'çatal', 'bıçak', 'kaşık', 'kase', 'muz', 'elma', 'sandviç', 'portakal', 'brokoli', 'havuç', 'hotdog'
          , 'pizza', 'donut', 'pasta', 'sandalye', 'kanepe', 'saksı bitkisi', 'yatak', 'yemek masası', 'tuvalet', 'televizyon', 'laptop', 'maus',
          'uzaktan kumanda', 'klavye', 'cep telefonu', 'mikrodalga', 'fırın', 'tost makinesi', 'lavabo', 'buzdolabı', 'kitap', 'saat', 'vazo', 'makas'
          , 'oyuncak ayı', 'saç kurutucu', 'diş fırçası']

canli = ['insan', 'kuş', 'kedi', 'köpek', 'at', 'koyun', 'inek', 'fil', 'ayı', 'zebra', 'zürafa']

# Webcam başlat
kamera = cv2.VideoCapture(0)  # 0, birinci kamera demektir, eğer birden fazla kamera varsa uygun numarayı kullanın
font = cv2.FONT_HERSHEY_SIMPLEX

try:
    while True:
        ret, kare = kamera.read()

        resimler = cv2.cvtColor(kare, cv2.COLOR_BGR2RGB)
        sonuclar = model(resimler, verbose=False)

        canli_nesne = False  # Varsayılan olarak canlı nesne yok

        for i in range(len(sonuclar[0].boxes)):
            x1, y1, x2, y2 = sonuclar[0].boxes.xyxy[i]
            skor = sonuclar[0].boxes.conf[i]
            etiket = sonuclar[0].boxes.cls[i]
            x1, y1, x2, y2, skor, etiket = (int(x1), int(y1), int(x2), int(y2), float(skor), int(etiket))
            nesne_ismi = etiketler[etiket]

            if skor < 0.5:
                continue

            if nesne_ismi in canli:
                canli_nesne = True

            cv2.rectangle(kare, (x1, y1), (x2, y2), (255, 0, 0), 2)
            yazi = nesne_ismi + ' ' + str(format(skor, '.2f'))
            cv2.putText(kare, yazi, (x1, y1 - 10), font, 0.55, (255, 0, 255), 2)

        # Ardunio'ya veriyi gönder
        if canli_nesne:
            ser.write(b'H')  # Canlı nesne varsa 'H' gönder
            print("Canlı Nesne Bulundu!")
        else:
            ser.write(b'L')  # Canlı nesne yoksa 'L' gönder
            print("Canlı Nesne Bulunamadı")

        cv2.imshow("Webcam", kare)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

except Exception as e:
    print(f"Hata: {e}")

finally:
    # Kullanılan kaynakları serbest bırak ve portu kapat
    kamera.release()
    cv2.destroyAllWindows()
    ser.write(b'L')  # Hata durumunda 'L' gönder
    ser.close()